# Zuck PDF Body Creator

*Eddie Chapman*

This program reads a folder of `.txt` files and outputs `.pdf` documents. It begins copying text after reading `##content##`. After, all names formatted as `##Name##` are bolded and hashtags removed. 

`Zuck Metadata Adder` must be run beforehand in order to add `##content##`. 

Output is `Record_ID-body.pdf`

### Set-up

Program uses `reportlab` for PDF generation and `chardet` for encoding stuff

In [127]:
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY
from chardet.universaldetector import UniversalDetector  

Setting local directory. Make sure it matches your desired folder!

In [128]:
os.chdir('C:\\Users\\chapman4\\Downloads\\zuck-text-ready-test\\text\\text-new')

### List Filenames

Creating a list of filenames using the folder of text files. Text files should be named the same thing. 
 
 `name.split()` removes the file extension. 

In [129]:
def list_filenames():
    filenames = [name.split(".")[0] for name in os.listdir(".") if name.endswith(".txt")]
    return filenames

### Determine Encoding 
Figuring out what encoding each text file is. I'm not super sure this helps the process. Hopefully everything is UTF-8 in the future...

In [130]:
def determine_encoding(filename):
    detector = UniversalDetector()
    with open(filename + '.txt', 'rb') as infile: 
        detector.reset()
        for line in infile:
            detector.feed(line)
            if detector.done:   # detection process ends automatically when confidence is high enough
                break
        detector.close()
        return detector.result

### Create PDF
Making the .pdf using the filename and encoding.  
  
Initializes a .pdf document (`SimpleDocTemplate` + minor tweeks from `styles.add`) 

Then it reads the text file line by line until it hits `##content##`, after which it copies the entire remaining document.

All hashtagged lines (`##sample##`) have their hashtags removed and are bolded. 

All non-hashtagged lines are formmated as paragraphs. 

The PDF is exported (`doc.build()`)

In [131]:
def create_pdf(filename, detector_result):
    with open(filename + '.txt', 'r', encoding = detector_result['encoding']) as infile:  
        doc = SimpleDocTemplate(filename + '_body.pdf', pagesize=letter, rightMargin=80, leftMargin=80,\
                                topMargin=60, bottomMargin=60)
        styles = getSampleStyleSheet()
        styles.add(ParagraphStyle(name='Paragraph', fontName='Helvetica', fontSize=12,  leading=18))
        styles.add(ParagraphStyle(name='Bold', fontName='Helvetica-Bold', fontSize=12))
            
        content = []
        
        for line in infile:
            if '##content##' in line:
                for line in infile:
                    if not line.isspace():
                        clean_line = line.strip()
                        if '##' in line:
                            tag_line = clean_line.replace('##', '')
                            uni_tag_line = tag_line.encode('utf-8')
                            content.append(Paragraph(uni_tag_line, styles["Bold"]))
                            content.append(Spacer(1, 12))
                        else:
                            uni_line = clean_line.encode('utf-8')
                            content.append(Paragraph(uni_line, styles["Paragraph"]))
                            content.append(Spacer(1, 12))
        doc.build(content) 

### Order of Operations    

In [132]:
def main():
    for filename in list_filenames():
        detector_result = determine_encoding(filename)
        create_pdf(filename, detector_result)

In [133]:
main()